In [ ]:
#!/bin/bash
# https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri/data
# !kaggle datasets download sartajbhuvaji/brain-tumor-classification-mri
# !unzip "./brain-tumor-classification-mri.zip"


In [ ]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, precision_score
import numpy as np
import torchvision.transforms as v2 
import albumentations as A
import mlflow
from aux import ImagesMRIDataset, split_for_cross_validation, get_training_testing_data, split_traing_data 

class EarlyStopping():
    def __init__(self, path : str, patience=5, threshold=1e-4):
        self.patience = patience
        self.threshold = threshold
        self.min_loss = 10000
        self.steps_till_stop = 0
        self.path = path

    def continue_training(self, model, loss):
        if(loss < self.min_loss - self.threshold):
            self.min_loss = loss
            self.steps_till_stop = 0
            torch.save(model.state_dict(), self.path)
            return True
        if (loss >= self.min_loss - self.threshold):
            self.steps_till_stop += 1
            if (self.steps_till_stop == self.patience): return False
        return True
    
    def load_model(self, model):
        model.load_state_dict(torch.load(self.path, weights_only=True))
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        return model


In [ ]:
SIZE = (100, 100)
train_transform = A.Compose([
    A.Resize(height=SIZE[0], width=SIZE[1]),  
    A.RandomGamma(gamma_limit=(80, 120), p=1.0), 
    A.HorizontalFlip(p = 0.5),
    # A.ShiftScaleRotate(p = 0.5),
    A.CLAHE(clip_limit=5.0, tile_grid_size=(8, 8), p=1.0), 
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.8), 
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

test_transformations = A.Compose([
    A.Resize(height=SIZE[0], width=SIZE[1]),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

def compute_train_transformations(image):
    image = train_transform(image=image)["image"]
    return np.array(image.transpose((2, 0, 1)), dtype=np.float32)
    
def compute_test_transformations(image):
    image = test_transformations(image=image)["image"]
    return np.array(image.transpose((2, 0, 1)), dtype=np.float32)


In [ ]:
def training_loop(model, criterion, optimizer, dataloader : DataLoader):
   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   current_training_loss = 0
   all_train_labels, all_train_preds = [], []
   model.train()
   for idx, (images, labels) in enumerate(dataloader):
      images, labels = images.to(device), labels.to(device)
      output = model(images)
      output = output.to(device)
      loss = criterion(output, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      current_training_loss += loss.item()
      all_train_preds.extend(output.argmax(dim=1).cpu().numpy())
      all_train_labels.extend(labels.cpu().numpy())
   loss = current_training_loss / len(dataloader)
   acc = round(accuracy_score(all_train_preds, all_train_labels), 3)
   return loss, acc

def validation_loop(model, criterion, dataloader : DataLoader):
   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   all_val_labels, all_val_preds = [], [],
   current_validation_loss = 0
   model.eval()
   with torch.no_grad():
      for idx, (images, labels) in enumerate(dataloader):
         images, labels = images.to(device), labels.to(device)
         output = model(images)
         output = output.to(device)
         loss = criterion(output, labels)
         current_validation_loss += loss.item()
         all_val_labels.extend(labels.cpu().numpy())
         all_val_preds.extend(output.argmax(dim=1).cpu().numpy())
   
   loss = current_validation_loss / len(dataloader)
   acc =  round(accuracy_score(all_val_labels, all_val_preds), 3)
   precision = precision_score(all_val_preds, all_val_labels, average='macro')
   recall = recall_score(all_val_preds, all_val_labels, average='macro')
   f1 = f1_score(all_val_preds, all_val_labels, average='macro')
   return loss, acc, precision, recall, f1  

def train_model(parameters, model, data):
  earlyStopping : EarlyStopping = parameters["early_stopping"]
  current_acc = 0.0
  for i in range(parameters["epochs"]):
      tloss, tacc = training_loop(model, parameters["criterion"], parameters["optimizer"], data["train"])
      mlflow.log_metrics(
      metrics={
            "train_loss": tloss,
            "train_accuracy": tacc
         }
      )
      vloss, vacc, vprecision, vrecall, vf1 = validation_loop(model, parameters["criterion"], data["validation"])
      mlflow.log_metrics(
      metrics={
          "validation_loss": vloss,
          "validation_accuracy": vacc,
          "validation_precision": vprecision,
          "validation_recall": vrecall,
          "validation_f1": vf1,
          }
      )
      if (earlyStopping != None and not earlyStopping.continue_training(model, vloss)):
         print("Acuratetea nu a crescut de ceva vreme, a intervenit early stopping")
         break
      if (vacc > current_acc): 
         current_acc = vacc
         print(f"Acuratetea maxima {vacc}, la epoca {i}, loss de {vloss}")
      else:
         print(f"acuratete {vacc}, epoca {i}, loss {vloss}")

In [ ]:
class Net(nn.Module):
    def __init__(self,  width : int, expansion : int):
        super().__init__()
        self.relu = nn.ReLU()
        self.width1 = width
        self.conv1 = nn.Conv2d(3, self.width1, kernel_size=(3, 3))
        self.bn1 = nn.BatchNorm2d(self.width1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.width2 = self.width1 * expansion
        self.conv2 = nn.Conv2d(self.width1, self.width2, kernel_size=(3, 3))
        self.bn2 = nn.BatchNorm2d(self.width2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.width3 = self.width2 * expansion
        self.conv3 = nn.Conv2d(self.width2, self.width3, kernel_size=(5, 5))
        self.bn3 = nn.BatchNorm2d(self.width3)
        self.maxpool3 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.width4 = self.width3 * expansion
        self.conv4 = nn.Conv2d(self.width3, self.width4, kernel_size=(3, 3))
        self.bn4 = nn.BatchNorm2d(self.width4)
        self.maxpool4 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.width5 = self.width4 * expansion
        self.conv5 = nn.Conv2d(self.width4, self.width5, kernel_size=(3, 3))
        self.bn5 = nn.BatchNorm2d(self.width5)

        # (1024, 12) -> (1024, 1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classification_layer = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear(self.width5, self.width4),
            nn.Dropout(p=0.5),
            nn.Linear(self.width4, self.width3),
            nn.Dropout(p=0.5),
            nn.Linear(self.width3, 4),
            # nn.Softmax(dim=1)
            nn.LogSoftmax(dim=1),
        )
        
    def forward(self, x):
        x = self.maxpool1(self.relu(self.bn1(self.conv1(x))))
        x = self.maxpool2(self.relu(self.bn2(self.conv2(x))))
        x = self.maxpool3(self.relu(self.bn3(self.conv3(x))))
        x = self.maxpool4(self.relu(self.bn4(self.conv4(x))))
        x = self.avgpool(self.relu(self.bn5(self.conv5(x))))
        return self.classification_layer(x)

In [ ]:
def create_cross_validation_data(parameters, data_chunks, test_data, current : int):
    training_data = []
    for idx, chunk in enumerate(data_chunks):
        if (idx == current):
            continue
        training_data += chunk
    return {
        "train": DataLoader(ImagesMRIDataset(training_data, transformations=parameters["train_transform"]), batch_size=parameters["batch_size"], shuffle=True, drop_last=True),
        "validation": DataLoader(ImagesMRIDataset(data_chunks[current], transformations=parameters["test_transform"]), batch_size=parameters["batch_size"], shuffle=True),
        "test": DataLoader(ImagesMRIDataset(test_data, transformations=parameters["test_transform"]), batch_size=parameters["batch_size"], shuffle=True)
    }

def compute_cross_validation(parameters, K, data_chunks, test_info):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    idx = 0
    with mlflow.start_run(run_name=f"fold_{idx}") as run:
        model = Net(width=parameters["width"], expansion=parameters["expansion"])
        parameters["optimizer"] = torch.optim.Adam(model.parameters(), lr=parameters["lr"], weight_decay=parameters["weight_decay"])
        model = torch.jit.script(model)
        model = model.to(device)
        mlflow.log_params(parameters)
        data = create_cross_validation_data(parameters, data_chunks, test_info, idx)
        train_model(parameters, model, data)
        if (parameters["early_stopping"]  != None):
            best_model = Net(width=parameters["width"], expansion=parameters["expansion"])
            parameters["early_stopping"].load_model(best_model)
        else:
            best_model = model 
        loss, acc, precision, recall, f1 = validation_loop(best_model, parameters["criterion"], data["test"])
        mlflow.log_metrics(
            metrics={
          "test_loss": loss,
          "test_accuracy": acc,
          "test_precision": precision,
          "test_recall": recall,
          "test_f1": f1,
          }
        ) 
        print(f"pentru test: acuratete {acc}, pierderea este {loss}")

In [53]:
from collections import Counter

K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)

training_no_transforms = v2.Compose([
    v2.ToTensor(),
    v2.Resize(SIZE)
])

# EarlyStopping(path="./aici.pth", patience=5)
parameters = {
        "batch_size": 50,
        "epochs" : 10,
        "lr": 1e-3,
        "weight_decay": 1e-4,
        "width": 4,
        "expansion" : 2,
        "train_transform": training_no_transforms,
        "test_transform": training_no_transforms,
        "criterion": torch.nn.CrossEntropyLoss(),
        "optimizer": None,
        "scheduler": None,
        "early_stopping": None,
    }

# task 1 - fara augmentari, duplicarea exemplelor din clasele suplimentare
mlflow.set_experiment(experiment_name="Task1")
compute_cross_validation(parameters, K, data_chunks, test_info)

Acuratetea maxima 0.504, la epoca 0, loss de 1.130616762421348
Acuratetea maxima 0.685, la epoca 1, loss de 0.7889030738310381
Acuratetea maxima 0.743, la epoca 2, loss de 0.6478314860300585
Acuratetea maxima 0.821, la epoca 3, loss de 0.5002362050793387
acuratete 0.762, epoca 4, loss 0.6370058113878424
Acuratetea maxima 0.832, la epoca 5, loss de 0.4426937699317932
Acuratetea maxima 0.86, la epoca 6, loss de 0.4337602122263475
acuratete 0.845, epoca 7, loss 0.4607644582336599
acuratete 0.809, epoca 8, loss 0.5534026514400135
acuratete 0.858, epoca 9, loss 0.48577804592522705
pentru test valoarea de pierdere 1.6788892894983292 si cea de acuratete 0.706


In [55]:
K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)

# task 2 - cu augmentari, cu duplicarea exemplelor din clasele suplimentare
parameters = {
        "batch_size": 50,
        "epochs" : 10,
        "lr": 1e-3,
        "weight_decay": 1e-4,
        "width": 4,
        "expansion" : 2,
        "train_transform": compute_train_transformations,
        "test_transform": compute_test_transformations,
        "criterion": torch.nn.CrossEntropyLoss(),
        "optimizer": None,
        "scheduler": None,
        "early_stopping": None,
    }

mlflow.set_experiment(experiment_name="Task2")
compute_cross_validation(parameters, K, data_chunks, test_info)

/home/alf/anul4/anul4_tot/IS/InvatareSupervizataTema2/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Acuratetea maxima 0.374, la epoca 0, loss de 1.3199936693364924
Acuratetea maxima 0.513, la epoca 1, loss de 1.0807893655516885
Acuratetea maxima 0.543, la epoca 2, loss de 1.0228559320623225
Acuratetea maxima 0.643, la epoca 3, loss de 0.8228958628394387
acuratete 0.606, epoca 4, loss 0.9901242635466836
acuratete 0.626, epoca 5, loss 0.8829161308028481
Acuratetea maxima 0.657, la epoca 6, loss de 0.7653163888237693
Acuratetea maxima 0.74, la epoca 7, loss de 0.6770976077426564
acuratete 0.67, epoca 8, loss 0.7841290018775247
acuratete 0.691, epoca 9, loss 0.7694819948889993
pentru test: acuratete 0.518, pierderea este 1.433277890086174


In [ ]:
K = 5
BALANCED = False
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)

def compute_class_weights(data_stream):
    labels = [label for _, label in data_stream]
    label_counts = Counter(labels)
    total_samples = sum(label_counts.values())
    class_weights = {label: total_samples / count for label, count in label_counts.items()}
    max_label = max(label_counts.keys())
    weights_list = [class_weights.get(i, 0.0) for i in range(max_label + 1)]
    weights = torch.tensor(weights_list, dtype=torch.float)
    print(weights)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    weights = weights.to(device)
    return weights 


def compute_functions_criterion_weighted(model : Net, parameters):
    optimizer = torch.optim.Adam(model.parameters(), lr=parameters["lr"], weight_decay=parameters["weight_decay"])
    return torch.nn.CrossEntropyLoss(weight=compute_class_weights(training_info)), optimizer, None, None

# Task 2 cu augmentari - cu functia de loss avand weights
task2_augmentation_test_weighted, task2_augmentation_validation_weighted = compute_cross_validation(current_parameters, K, data_chunks, test_info, compute_train_transformations, compute_test_transformations, compute_functions_criterion_weighted)

In [ ]:
from monai.transforms import Resize, Rand2DElastic, RandAdjustContrast, RandRotate, ScaleIntensity, Compose
from monai.transforms import RandGaussianNoise, RandAffine, RandStdShiftIntensity
from monai.transforms import RandGaussianSmooth, ThresholdIntensity, RandAxisFlip
from monai.transforms import RandSpatialCrop, RandGaussianSharpen, RandHistogramShift, RandShiftIntensity
import cv2

first_augmentation_pipeline = Compose([
    Resize(spatial_size= SIZE),
    RandGaussianSharpen(),
    RandAxisFlip(prob=0.3),
    RandSpatialCrop(roi_size=SIZE, random_center=True, random_size=False),
    RandRotate(range_x=(-50, 50),  prob=0.5, keep_size=True),
    RandAdjustContrast(prob=0.8),
    ScaleIntensity(minv=0.0, maxv=1.0)
])

second_augmentation_pipeline = Compose([
    Resize(spatial_size= SIZE),
    Rand2DElastic(spacing=(5, 10), magnitude_range=(1, 2), prob=0.5, rotate_range=(0, 0), shear_range=(0.02, 0.02),
                    translate_range=(10, 10), scale_range=(0.1, 0.1), spatial_size=SIZE, mode="bilinear", padding_mode="reflection"),
    RandHistogramShift(prob=0.8),
    ThresholdIntensity(threshold=20),
    RandGaussianSmooth(sigma_x=(0.5, 1.5), sigma_y=(0.5, 1.5), prob=0.8),
    RandRotate(range_x=(-50, 50),  prob=0.5, keep_size=True),
    ScaleIntensity(minv=0.0, maxv=1.0)
])


third_augmentation_pipeline = Compose([
    Resize(spatial_size= SIZE),
    RandGaussianNoise(mean=0.0, std=0.1, prob=0.8),
    RandShiftIntensity(offsets=(-20, 20), prob=0.3, safe=True),
    RandAxisFlip(prob=0.3),
    RandStdShiftIntensity(factors=0.8),
    RandAffine(prob=0.2),
    ScaleIntensity(minv=0.0, maxv=1.0)
])

test_augmentation_pipeline = Compose([
    Resize(spatial_size= SIZE),
    ScaleIntensity(minv=0.0, maxv=1.0)
])

def create_test_monai(img):
    img = np.transpose(img, (2, 0, 1))
    return test_augmentation_pipeline(img)

def create_first_monai_transform(img):
    img = np.transpose(img, (2, 0, 1))
    return first_augmentation_pipeline(img)


def create_second_monai_transform(img):
    img = np.transpose(img, (2, 0, 1))
    return second_augmentation_pipeline(img)


def create_third_monai_transform(img):
    img = np.transpose(img, (2, 0, 1))
    return third_augmentation_pipeline(img)

In [ ]:
K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)
task3_first_monai_test, task3_first_monai_validation = compute_cross_validation(current_parameters, K, data_chunks, test_info, create_first_monai_transform, create_test_monai, compute_functions_balanced_no_scheduler)

In [ ]:
task3_second_monai_test, task3_second_monai_validation = compute_cross_validation(current_parameters, K, data_chunks, test_info, create_second_monai_transform, create_test_monai, compute_functions_balanced_no_scheduler)

In [ ]:
task3_third_monai_test, task3_third_monai_validation = compute_cross_validation(current_parameters, K, data_chunks, test_info, create_third_monai_transform, create_test_monai, compute_functions_balanced_no_scheduler)

In [ ]:
def compute_functions_with_early_stopper(model : Net, parameters):
    optimizer = torch.optim.Adam(model.parameters(), lr=parameters["lr"], weight_decay=parameters["weight_decay"])
    return torch.nn.CrossEntropyLoss(), optimizer, None, EarlyStopping("./aici.pth", parameters['early_stopping_patience'])

def compute_functions_simple(model : Net, parameters):
    optimizer = torch.optim.Adam(model.parameters(), lr=parameters["lr"], weight_decay=parameters["weight_decay"])
    return torch.nn.CrossEntropyLoss(), optimizer, torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', 
                patience=parameters["optimizer_patience"], factor=parameters["lr_factor"]), EarlyStopping("./aici.pth", parameters['early_stopping_patience'])




In [ ]:
K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)
current_parameters = {
    "batch_size": 50,
    "epochs" : 50,
    "lr": 1e-3,
    "weight_decay": 1e-4,
    "optimizer_patience" : 3,
    "lr_factor": 0.6,
    "width": 16,
    "expansion" : 4,
    "early_stopping_patience" : 7,
}

task4_ealry_stopping_test, task4_ealry_stopping_validation = compute_cross_validation(current_parameters, K, data_chunks, test_info, create_third_monai_transform, create_test_monai, compute_functions_with_early_stopper)

In [ ]:
K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)
current_parameters = {
    "batch_size": 50,
    "epochs" : 25,
    "lr": 1e-3,
    "weight_decay": 1e-4,
    "optimizer_patience" : 3,
    "lr_factor": 0.6,
    "width": 16,
    "expansion" : 4,
    "early_stopping_patience" : 7,
}

def compute_functions_with_lr_scheduler(model : Net, parameters):
    optimizer = torch.optim.Adam(model.parameters(), lr=parameters["lr"], weight_decay=parameters["weight_decay"])
    return torch.nn.CrossEntropyLoss(), optimizer, torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', 
                patience=parameters["optimizer_patience"], factor=parameters["lr_factor"]), None


task4_lr_scheduler_test, task4_lr_scheduler_validation = compute_cross_validation(current_parameters, K, data_chunks, test_info, create_first_monai_transform, create_test_monai, compute_functions_with_lr_scheduler)

In [ ]:
K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)
parameters = {
    "batch_size": 16,
    "epochs" : 50,
    "lr": 1e-2,
    "weight_decay": 1e-4,
    "optimizer_patience" : 3,
    "lr_factor": 0.6,
    "width": 16,
    "expansion" : 4,
    "early_stopping_patience" : 7,
}

# 72 maxim
def compute_ablatiations(model : Net, parameters):
    # optimizer = torch.optim.Adam(model.parameters(), lr=parameters["lr"], weight_decay=parameters["weight_decay"])
    optimizer = torch.optim.SGD(model.parameters(), lr=parameters["lr"], weight_decay=parameters["weight_decay"], momentum=0.7)
    return torch.nn.BCEWithLogitsLoss(), optimizer, torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', 
                patience=parameters["optimizer_patience"], factor=parameters["lr_factor"]), EarlyStopping("./aici.pth", patience=parameters["early_stopping_patience"])

SIZE=(100, 100)
ablatiation_augmentation = Compose([
    Resize(spatial_size= SIZE),
    RandGaussianSharpen(),
    RandAxisFlip(prob=0.3),
    RandSpatialCrop(roi_size=SIZE, random_center=True, random_size=False),
    RandRotate(range_x=(-50, 50),  prob=0.5, keep_size=True),
    RandAdjustContrast(prob=0.8),
    ScaleIntensity(minv=0.0, maxv=1.0)
])


def create_ablatiation_transform(img):
    img = np.transpose(img, (2, 0, 1))
    return ablatiation_augmentation(img)


test_augmentation_ablatiation = Compose([
    Resize(spatial_size= SIZE),
    ScaleIntensity(minv=0.0, maxv=1.0)
])

def create_ablatiation_test_monai(img):
    img = np.transpose(img, (2, 0, 1))
    return test_augmentation_ablatiation(img)


validation_info = {"precision": [], "recall": [], "f1Score" : [], "accuracy" : []} 
info_test = {"precision": [], "recall": [], "f1Score" : [], "accuracy" : []}
for i in range(K):
    if (i != 1):
        continue
    model = Net(width=parameters["width"], expansion=parameters["expansion"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion, optimizer, lr_scheduler, early_stopping = compute_ablatiations(model, parameters)
    data = create_cross_validation_data(data_chunks, test_info, i, create_ablatiation_transform, create_ablatiation_test_monai, parameters["epochs"])
    training_loss, validation_loss, training_accuracy, validation_accuracy = train_model(model, parameters["epochs"],
            data, criterion, optimizer, lr_scheduler, early_stopping)
    if (early_stopping != None):
        best_model = Net(width=parameters["width"], expansion=parameters["expansion"])
        early_stopping.load_model(best_model)
    else:
        best_model = model
    precision, recall, f1, acc, mat = test_model(best_model, data["validation"], criterion)
    precision, recall, f1, acc, mat = test_model(best_model, data["test"], criterion)